In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Verileri oku
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

# Hedef değişken
y = train['Survived']

# Feature engineering için birleştir
combine = pd.concat([train.drop(columns=['Survived']), test], sort=False).reset_index(drop=True)

# Kategorik dönüşümler
combine['Sex'] = combine['Sex'].map({'male': 0, 'female': 1})
combine['Embarked'] = combine['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Eksik verileri doldur (uyarısız yöntemle)
combine['Age'] = combine['Age'].fillna(combine['Age'].median())
combine['Fare'] = combine['Fare'].fillna(combine['Fare'].median())
combine['Embarked'] = combine['Embarked'].fillna(combine['Embarked'].mode()[0])

# Kullanılacak özellikler
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

# Train ve test ayır
X_train = combine.loc[:len(y)-1, features]
X_test = combine.loc[len(y):, features]

# Model eğitimi
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y)

# Performans kontrolü
score = cross_val_score(model, X_train, y, cv=5)
print(f"CV Accuracy: {score.mean():.4f}")

# Tahmin ve gönderim dosyası
predictions = model.predict(X_test)
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})
submission.to_csv('submission.csv', index=False)


CV Accuracy: 0.8137
